In [1]:
# Input info
groups_f = ""
input_params = ""

outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/clonal_shifts/clones/"

clone_col = "name"
atac_col = "cluster_labels"


# params
p_thresh = 0.1 

In [2]:
import pandas as pd
import numpy as np
from os.path import join, exists
import os
from tqdm.notebook import tqdm

from scipy.stats import hypergeom, fisher_exact
from statsmodels.stats import multitest 

import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from src import clonal_shifts as cs
from icecream import ic

### Setup outdirs

In [ ]:
sim_dir = join(outdir, "shuffle")
if not exists(sim_dir):
    os.mkdir(sim_dir)

## Load groups

In [ ]:
groups = pd.read_csv(groups_f, sep="\t")
sizes, clones, atac_cl, clone_col, atac_col = pickle.load(open(input_params, "rb"))

## Run

In [ ]:
clone_map = {x:ind for ind,x in enumerate(clones)}

output_df, bh_enrichment_df = cs.pipeline_groups_hypergeo(groups, clones, atac_cl, sizes,p_thresh, atac_col, clone_col)
bh_enrichment_df.to_csv(join(outdir, "hypergeo_padjusted.csv"))
output_df.to_csv(join(outdir, "hypergeo_padjusted_sigOnly.csv"))
    
g = sns.clustermap(-np.log10(bh_enrichment_df.loc[output_df.index].fillna(1)), 
               row_cluster=False)

g.ax_heatmap.set(xlabel="Cluster ID")
g.ax_cbar.set(title="-log10 p-value")
plt.savefig(join(outdir, "hypergeo_padjusted_sigOnly.png"))



In [ ]:
## Run shuffle
init_bh_enrichment = cs.create_enrichment(groups, atac_col, clone_col, p_thresh,
                                          clones=clones, atac_cl=atac_cl)

shuffle = cs.shuffle_hypergeo(groups, atac_col, clone_col, p_thresh, clones, atac_cl, n_shuffle=1000, 
                              to_parallel=True, n_cpus=24)

out_all,out_min, out_cloneall, out_clonemin = cs.get_out(shuffle, clones, bh_enrichment_df, p_thresh, clone_map, atac_col, 
                                                      outdir=sim_dir)